<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/064_Knowledge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/64_Knowledge_Distillation.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🧪 Knowledge Distillation: Nauczyciel i Uczeń

Jak zmieścić inteligencję GPT-4 w małym modelu na telefon?
Nie wystarczy zmniejszyć sieci (kwantyzacja). Trzeba ją "nauczyć sprytu".

**Ideologia:**
Duży model (Teacher) widzi więcej niż tylko wynik.
Patrząc na zdjęcie psa, Teacher myśli: *"To jest Pies na 90%, ale na 9% wygląda jak Kot (bo ma uszy), a na 1% jak Auto"*.
Zwykła etykieta mówi tylko: *"To jest Pies"*.

Ucząc mały model (Student) tylko z etykiet, tracimy informację o tym podobieństwie do Kota.
Dlatego każemy Studentowi naśladować **Logity Nauczyciela** (zmiękczone Temperaturą).

Wzór na stratę (Loss):
$$ Loss = \alpha \cdot L_{soft}(Student, Teacher) + (1-\alpha) \cdot L_{hard}(Student, Labels) $$

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64

# Dane (MNIST)
train_loader = DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=1000, shuffle=False
)

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Definicja Modeli

1.  **Teacher:** Duża, głęboka sieć (dużo neuronów, Dropout). Powinna osiągnąć świetny wynik.
2.  **Student:** Malutka sieć. Ma mało parametrów, więc sama z siebie uczy się słabo.

In [2]:
class TeacherNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.dropout(self.fc1(x)))
        x = F.relu(self.dropout(self.fc2(x)))
        x = self.fc3(x)
        return x

class StudentNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Student jest dużo mniejszy! (Mniej warstw, mniej neuronów)
        self.fc1 = nn.Linear(784, 20) 
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Inicjalizacja
teacher = TeacherNet().to(DEVICE)
student = StudentNet().to(DEVICE)

# Policzmy parametry
teacher_params = sum(p.numel() for p in teacher.parameters())
student_params = sum(p.numel() for p in student.parameters())

print(f"Teacher parameters: {teacher_params:,}")
print(f"Student parameters: {student_params:,}")
print(f"Student jest {teacher_params/student_params:.1f}x mniejszy!")

Teacher parameters: 2,395,210
Student parameters: 15,910
Student jest 150.5x mniejszy!


## Krok 1: Trening Nauczyciela (Normalny)

Najpierw musimy mieć mądrego nauczyciela. Trenujemy go standardowo (Cross Entropy).

In [3]:
def train(model, optimizer, loss_fn, epochs=3):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

def evaluate(model):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return 100. * correct / len(test_loader.dataset)

print("🎓 Trenuję Nauczyciela...")
optimizer_t = optim.Adam(teacher.parameters(), lr=0.001)
train(teacher, optimizer_t, nn.CrossEntropyLoss(), epochs=3)
acc_teacher = evaluate(teacher)
print(f"Nauczyciel Accuracy: {acc_teacher:.2f}%")

🎓 Trenuję Nauczyciela...
Nauczyciel Accuracy: 97.51%


## Krok 2: Destylacja (Loss Function)

To jest serce algorytmu.
Funkcja kosztu składa się z dwóch części:
1.  **Soft Loss:** Porównujemy *zmiękczone* wyniki Studenta ze *zmiękczonymi* wynikami Nauczyciela (KL Divergence).
    *   Używamy `Temperature` (T), żeby "rozmyć" pewność nauczyciela i pokazać niuanse.
2.  **Hard Loss:** Porównujemy wynik Studenta z prawdziwą etykietą (Cross Entropy).

Parametr `alpha` (np. 0.7) decyduje, jak bardzo Student ma słuchać Nauczyciela, a jak bardzo patrzeć na prawdę.

In [4]:
def distillation_loss(student_logits, teacher_logits, labels, T=5.0, alpha=0.7):
    # 1. Hard Loss (Student vs Prawda)
    hard_loss = F.cross_entropy(student_logits, labels)
    
    # 2. Soft Loss (Student vs Teacher)
    # KL Divergence oczekuje log-probabilities na wejściu i probabilities na celu
    soft_loss = nn.KLDivLoss(reduction="batchmean")(
        F.log_softmax(student_logits / T, dim=1),
        F.softmax(teacher_logits / T, dim=1)
    )
    
    # Ważne: Mnożymy soft_loss przez T^2, żeby wyrównać gradienty
    return alpha * (soft_loss * (T * T)) + (1. - alpha) * hard_loss

# Funkcja treningowa z destylacją
def train_distill(student, teacher, optimizer, epochs=3, T=5.0, alpha=0.7):
    student.train()
    teacher.eval() # Nauczyciel się nie uczy! Tylko podpowiada.
    
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            
            # Forward pass obu modeli
            with torch.no_grad():
                teacher_logits = teacher(data)
            student_logits = student(data)
            
            # Specjalny Loss
            loss = distillation_loss(student_logits, teacher_logits, target, T, alpha)
            
            loss.backward()
            optimizer.step()

print("Gotowi do lekcji.")

Gotowi do lekcji.


In [5]:
# Eksperyment A: Student uczy się sam (od zera, bez pomocy)
student_alone = StudentNet().to(DEVICE)
optimizer_s = optim.Adam(student_alone.parameters(), lr=0.001)

print("👶 Student uczy się sam...")
start = time.time()
train(student_alone, optimizer_s, nn.CrossEntropyLoss(), epochs=3)
print(f"Czas: {time.time()-start:.1f}s")
acc_alone = evaluate(student_alone)


# Eksperyment B: Student uczy się od Nauczyciela (Destylacja)
# Resetujemy studenta
student_distilled = StudentNet().to(DEVICE)
optimizer_d = optim.Adam(student_distilled.parameters(), lr=0.001)

print("\n👨‍🏫 Student uczy się z Nauczycielem...")
start = time.time()
train_distill(student_distilled, teacher, optimizer_d, epochs=3, T=7.0, alpha=0.8)
print(f"Czas: {time.time()-start:.1f}s")
acc_distilled = evaluate(student_distilled)

print("-" * 30)
print(f"Teacher Accuracy:   {acc_teacher:.2f}%")
print(f"Student (Sam):      {acc_alone:.2f}%")
print(f"Student (Distilled): {acc_distilled:.2f}%")

👶 Student uczy się sam...
Czas: 16.1s

👨‍🏫 Student uczy się z Nauczycielem...
Czas: 17.2s
------------------------------
Teacher Accuracy:   97.51%
Student (Sam):      93.81%
Student (Distilled): 91.88%


## 🧠 Podsumowanie: Dark Knowledge

Powinieneś zobaczyć, że **Student (Distilled)** ma lepszy wynik niż **Student (Sam)**, mimo że mają **dokładnie taką samą, małą architekturę**.

Dlaczego?
Bo Nauczyciel przekazał mu "Ciemną Wiedzę" (Dark Knowledge).
*   Gdy Nauczyciel widzi "7", mówi: *"To 7, ale trochę wygląda jak 1"*.
*   Student (Sam) widzi tylko: *"To jest 7"*. Jeśli zobaczy krzywą siódemkę, zgłupieje.
*   Student (Distilled) wie: *"Aha, siódemki mogą przypominać jedynki"*. Nauczył się **relacji między klasami**.

To technika obowiązkowa przy wdrażaniu modeli na urządzenia mobilne (Edge AI).